### Prepare Data
- convert categorical fields to dummies
- normalize data
- test/train/validation set split

### Train and Evaluate
- Train the model using 1) RandomForest and 2) XGBoost
- Evaluate the predictions agains the test set

### Prediction API
- Implement a function that takes raw data as input and produces the hourly predictions
    - implement a function that converts the dummies back to categorical
    - denormalize the data back to the original scale    